In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import arviz as az

WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
model = pm.Model()
with model:
    alpha = 4.66
    clients_per_hour = pm.Poisson('clients_per_hour' ,alpha)
    waiting_time = pm.Normal('waiting_time', mu=1, sigma=1/2)
    avg_order = pm.Exponential('avg_order_time', 1/alpha)
    trace = pm.sample(20000)

dictionary = {
              'preparation_time': trace['avg_order_time'].tolist(),
              'waiting_time': trace['waiting_time'].tolist(),
              }
df = pd.DataFrame(dictionary)



/srv/conda/envs/notebook/lib/python3.7/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Multiprocess sampling (2 chains in 2 jobs)
CompoundStep
>Metropolis: [clients_per_hour]
>NUTS: [avg_order_time, waiting_time]


Sampling 2 chains for 1_000 tune and 20_000 draw iterations (2_000 + 40_000 draws total) took 59 seconds.
There were 253 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.706646380867502, but should be close to 0.8. Try to increase the number of tuning steps.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 25% for some parameters.


In [19]:
waiting_time_under_15_mins = df[(df['waiting_time'] + df['preparation_time'] <= 15)]

sample_size =  df.shape[0]

waiting_time_under_15_mins_percentage = waiting_time_under_15_mins.shape[0] / sample_size
print("Waiting time under 15 minutes percentage", waiting_time_under_15_mins_percentage)

Waiting time under 15 minutes percentage 0.948575


In [20]:
customer_waiting_time = list(sum(x) for x in zip(df['waiting_time'], df['preparation_time']))
average_customer_waiting_time = sum(customer_waiting_time) / sample_size

print("Average customer waiting time:", average_customer_waiting_time)

Average customer waiting time: 5.7208976742899775


In [ ]:

az.plot_posterior({"Clients per hour": trace['clients_per_hour'], "Waiting Time" : trace['waiting_time'], "Avg order time" : trace['avg_order_time']})
    